In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.express as px
import torch
import umap
from loguru import logger
from nltk.tokenize import sent_tokenize
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
df_path = Path("data") / "raw" / "geo-reviews-dataset-2023.csv"
df = pd.read_csv(df_path)
df

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.0,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.0,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.0,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.0,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.0,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...
...,...,...,...,...,...
499995,"Москва, Южный административный округ, район Би...",Бирюлёво-Пассажирская,4.0,Железнодорожная станция,"Охрана кривая но добрая, двери не закрываются ..."
499996,"Москва, Южный административный округ, район Би...",Бирюлёво-Пассажирская,4.0,Железнодорожная станция,По сравнению со многими современными платформа...
499997,"Новосибирск, Коммунистическая улица, 48А",NaN,5.0,"Бар, паб","Приятная атмосфера, прекрасное вино, волшебная..."
499998,"Астраханская область, Харабалинский район",Сарай-Бату,5.0,Достопримечательность,Был с семьёй 13.06.23 Отличное место. Рекоменд...


In [3]:
df[df["text"].str.startswith("Нормальная стоматология")]

,address,name_ru,rating,rubrics,text
169342,"Самарская область, Тольятти, улица Свердлова, 9",ГБУЗ Со Тольяттинская стоматологическая поликл...,5.0,Стоматологическая поликлиника;Поликлиника для ...,Нормальная стоматология лечил зубы на платной ...
179816,"Самара, Владимирская улица, 21",ГБУЗ Со Самарская стоматологическая поликлиник...,5.0,Стоматологическая поликлиника,Нормальная стоматология. Стоматолог -терапевт ...


In [18]:
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

In [20]:
model._model_config

{'__version__': {'sentence_transformers': '2.0.0',
  'transformers': '4.7.0',
  'pytorch': '1.9.0+cu102'}}

In [ ]:
def preprocess_text(text: str) -> str:
    text = text.replace("\n", " ")
    text = text.replace("\\n", " ")
    text = text.replace("\t", " ")
    text = text.replace("-", "")
    text = text.replace(";", " ")
    return text


def mean_pool_sentence_embeddings(texts: list[str], model: SentenceTransformer) -> np.ndarray:
    all_embeddings = []
    for text in texts:
        sentences = sent_tokenize(text)
        sentence_embeddings = model.encode(sentences, convert_to_numpy=True)
        mean_embedding = np.mean(sentence_embeddings, axis=0)
        all_embeddings.append(mean_embedding)
    return np.array(all_embeddings)


def average_precision_at_k(relevant_list: list[int], k: int) -> float:
    score = 0.0
    num_hits = 0
    for i, rel in enumerate(relevant_list[:k]):
        if rel:
            num_hits += 1
            score += num_hits / (i + 1)
    return score / min(k, sum(relevant_list)) if sum(relevant_list) > 0 else 0.0


def benchmark_similarity(
    df: pd.DataFrame,
    doc_col: str,
    query_col: str,
    model: SentenceTransformer,
    k: int = 10,
) -> pd.DataFrame:
    if doc_col not in df.columns or query_col not in df.columns:
        logger.error(f"DataFrame must contain '{doc_col}' and '{query_col}' columns")
        raise ValueError(f"DataFrame must contain '{doc_col}' and '{query_col}' columns")

    df = df.copy()
    df[doc_col] = df[doc_col].apply(preprocess_text)
    df[query_col] = df[query_col].apply(preprocess_text)

    logger.info(f"Encoding docs, df.shape: {df.shape}")
    doc_embeddings = mean_pool_sentence_embeddings(df[doc_col].tolist(), model)

    ap_scores = []

    for query in df[query_col].unique():
        logger.info(f"Processing query: {query}")
        query_embedding = model.encode([query], convert_to_numpy=True)

        similarities = cosine_similarity(query_embedding, doc_embeddings)[0]

        top_k_indices = np.argsort(similarities)[::-1][:k]
        top_k_types = df.iloc[top_k_indices][query_col].tolist()

        for i in range(3):
            logger.info(f"Top {i} docs for query: {df.iloc[top_k_indices][doc_col].tolist()[i]}")

        relevant_list = [1 if t == query else 0 for t in top_k_types]
        ap_at_k = average_precision_at_k(relevant_list, k)
        ap_scores.append(ap_at_k)

    map_k = np.mean(ap_scores)

    return map_k


doc_col = "text"
query_col = "rubrics"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name, device=device)
benchmark_similarity(df.sample(2000), doc_col, query_col, model)

2025-04-12 16:06:30.891 | INFO     | __main__:benchmark_similarity:50 - Encoding docs, df.shape: (2000, 5)
2025-04-12 16:06:51.852 | INFO     | __main__:benchmark_similarity:56 - Processing query: Ресторан Пиццерия
2025-04-12 16:06:51.871 | INFO     | __main__:benchmark_similarity:65 - Top 0 docs for query: Отличный ресторан, приветливый персонал и очень вкусные блюда. 
2025-04-12 16:06:51.877 | INFO     | __main__:benchmark_similarity:65 - Top 1 docs for query: Неплохой, бюджетный ресторанчик.  Хорошее меню,весьма недурственные роллы, быстрое и четкое обслуживание,хорошая обстановка,дружнлюбные официанты. 
2025-04-12 16:06:51.878 | INFO     | __main__:benchmark_similarity:65 - Top 2 docs for query: Шикарное кафе. Вкусно очень, приветливый персонал. 
2025-04-12 16:06:51.878 | INFO     | __main__:benchmark_similarity:56 - Processing query: Ресторан Кафе
2025-04-12 16:06:51.914 | INFO     | __main__:benchmark_similarity:65 - Top 0 docs for query: Замечательная уютная атмосфера восточного

np.float64(0.10814864841004997)

In [3]:
df = pd.read_csv("data/raw/geo-reviews-dataset-2023.csv").sample(2000)
df

,address,name_ru,rating,rubrics,text
284668,"Москва, Зеленодольская улица, 32к3",Fix Price,5.0,Товары для дома;Магазин хозтоваров и бытовой х...,"Полный ассортимент, культурное обслуживание\n"
385425,"Тула, Серебровская улица, 16Е",InnDays,5.0,Гостиница,"Нужна была квартира на сутки, остановились на ..."
495627,"Санкт-Петербург, Красногвардейский район, исто...",Лидер,5.0,"Автосервис, автотехцентр","Отличный сервис. Грамотные специалисты , выпол..."
286204,"Республика Бурятия, Улан-Удэ, улица Балтахинов...",Гостиница Чингисхан,5.0,Гостиница,"Очень чисто, белье и полотенца белоснежные, в ..."
468236,"Ярославль, Которосльная набережная, 55",Royal Hotel,4.0,Гостиница,"Еда вполне вкусная, номера чистые, телевизор с..."
...,...,...,...,...,...
464284,"Москва, Мосфильмовская улица, 26",Домашний 24,5.0,Магазин продуктов;Супермаркет,"Благодарим за синьку по ночам, очень выручаете..."
42485,"Псковская область, Ольгинская набережная 5а",Кофейная коза,5.0,Кафе;Кофейня,"Очень приятное, комфортное и уютное место. Бол..."
41640,"Москва, Никольская улица, 17, стр. 1",Hello! Music,5.0,Музыкальное образование;Школа искусств,"Занимаюсь в школе уже полгода, есть прогресс, ..."
244832,"Москва, улица Сущёвский Вал, 5, стр. 1А",Савёловский,3.0,Торговый центр;Магазин электроники;Магазин одежды,"Плюсы: Можно купить всё, что угодно. Большой в..."


In [5]:
random_vectors = np.random.rand(2000, 256)

df["seara_test"] = list(random_vectors)

In [8]:
df.to_parquet("test.parquet", index=False)

In [11]:
df_1 = pd.read_parquet("test.parquet")

In [13]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   address     2000 non-null   object 
 1   name_ru     1998 non-null   object 
 2   rating      2000 non-null   float64
 3   rubrics     2000 non-null   object 
 4   text        2000 non-null   object 
 5   seara_test  2000 non-null   object 
dtypes: float64(1), object(5)
memory usage: 93.9+ KB


In [ ]:
df_1["seara_test"][0]

dtype('float64')

In [21]:
df = pd.read_parquet("data/embedding/geo-reviews-dataset-2023-with-embeddings.csv")
df

,address,name_ru,rating,rubrics,text,text_DeepPavlov/rubert-base-cased-sentence,text_sentence-transformers/distiluse-base-multilingual-cased-v1,text_sentence-transformers/distiluse-base-multilingual-cased-v2,text_sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2,text_sentence-transformers/paraphrase-multilingual-mpnet-base-v2
0,"Москва, Новопетровская улица, 16",Продукты,4.0,Магазин продуктов,"Неплохой магазин , есть фрукты, мясо и полуфаб...","[-0.50213087, -0.64826936, -0.17035411, -0.185...","[0.07690835, 0.0032311329, -0.002783761, 0.003...","[0.06391285, 0.015795773, -0.008594248, -0.002...","[0.04055049, -0.27353966, 0.12040212, 0.170736...","[-0.05627888, 0.07024487, -0.010127303, 0.0381..."
1,"Московская область, городской округ Пушкинский...",Акварель,5.0,Торговый центр,"Парковка удобная, есть на 1м этаже в самом ТЦ ...","[-0.60013384, -1.0376371, 0.45140484, -0.52933...","[0.040134322, 0.014427886, -0.020830415, -0.01...","[0.02186188, 0.0028836576, -0.017119097, -0.01...","[0.43145823, -0.065491855, -0.019453676, 0.081...","[-0.04901892, -0.093275495, -0.015669532, 0.04..."
2,"Псков, ул. Техническая / ул. Алексея Алёхина",Сити,5.0,Жилой комплекс,"Все нравится, кроме качества воды. Планировка ...","[-0.1946983, -1.0252259, 0.2142884, -0.5978085...","[-0.02027451, -0.03674739, 0.009350743, -0.041...","[-0.022215378, -0.027157225, 0.0011456553, -0....","[0.08608648, 0.013137772, -0.065643914, -0.034...","[-0.07400356, 0.006997604, -0.016366709, -0.05..."
3,"Тверь, проспект Победы, 3",Happy burger,5.0,Кафе Быстрое питание,"Туалет бесплатный, еду не пробовали, целью был...","[-0.63410574, -1.0250131, -0.010696959, -0.153...","[0.0050332267, 0.07289827, 0.007558107, 0.0310...","[-0.0302767, 0.050491307, 0.014834315, 0.02920...","[0.20306802, 0.048386086, -0.042062156, -0.137...","[0.10449143, 0.17446302, -0.015841514, -0.1183..."
4,"Ярославль, улица Кирова, 10/25",ЖаRим в КайF,4.0,Быстрое питание Кафе Ресторан,"Бургеры потрясающие, коктейли отличные, но над...","[-0.33710206, -0.7564586, -0.44192395, -0.1953...","[0.014471837, -0.014450337, 0.03563952, 0.0149...","[0.022243857, -0.013372567, 0.033722755, 9.218...","[0.07912045, -0.042511847, -0.13754338, 0.0355...","[-0.003875276, 0.11459597, -0.015715502, 0.012..."
...,...,...,...,...,...,...,...,...,...,...
95,"Самарская область, Тольятти, улица 40 лет Побе...",Мандариновый Краб,5.0,Ресторан Кафе,"Прекрасная атмосфера, все оформление выполнено...","[-0.24237785, -0.46768898, 0.0031302213, -0.40...","[-0.0071740584, -0.01002059, -0.020787051, -0....","[-0.01175545, -0.015101695, -0.0014910899, -0....","[0.15623124, -0.03778428, 0.083589554, 0.07880...","[-0.03158648, 0.034320123, -0.015918229, -0.00..."
96,"Московская область, Волоколамский городской ок...",Баня,5.0,Баня Сауна,"Меня зовут Николай, я пармастер, провожу индив...","[-0.40935588, -0.8374351, 0.12130034, -0.32133...","[0.009822574, -0.011381316, -0.021756353, -0.0...","[0.0042805537, -0.015936583, -0.013513985, -0....","[0.11982556, 0.11807666, 0.0065673753, 0.01375...","[-0.03292661, -0.08448532, -0.015476261, 0.019..."
97,"Московская область, городской округ Подольск, ...",Золотой Сазан,5.0,Клуб охотников и рыболовов,Шикарное место для отдыха и рыбалки! Приезжали...,"[-0.21206824, -0.24057968, -0.01653472, -0.109...","[-0.013622425, 0.0040116366, -0.014247133, -0....","[-0.008027322, -0.0045701824, -0.0051289126, -...","[0.14727236, 0.0819693, 0.09998082, -0.0707075...","[-0.03711533, 0.10205799, -0.017602654, -0.025..."
98,"Самарская область, Тольятти, Южное шоссе, 12, ...",Mehanick Shop,5.0,Магазин автозапчастей и автотоваров Автосервис...,Остался очень довольным качеством обслуживания...,"[-0.49992198, -0.9291309, 0.025501361, -0.2329...","[0.0027043293, 0.024749532, -0.025027702, -0.0...","[0.001730651, 0.025462046, -0.01284978, 0.0001...","[-0.0131317405, -0.029477526, 0.03312988, -0.1...","[-0.10191958, 0.035064716, -0.014480527, 0.042..."


In [1]:
df = pd.read_parquet("data/embedding/geo-reviews-dataset-2023-with-embeddings.parquet")
df

,address,name_ru,rating,rubrics,text,text_DeepPavlov/rubert-base-cased-sentence,text_sentence-transformers/distiluse-base-multilingual-cased-v1,text_sentence-transformers/distiluse-base-multilingual-cased-v2,text_sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2,text_sentence-transformers/paraphrase-multilingual-mpnet-base-v2
0,"Москва, Новопетровская улица, 16",Продукты,4.0,Магазин продуктов,"Неплохой магазин , есть фрукты, мясо и полуфаб...","[-0.50213087, -0.64826936, -0.17035411, -0.185...","[0.07690835, 0.0032311329, -0.002783761, 0.003...","[0.06391285, 0.015795773, -0.008594248, -0.002...","[0.04055049, -0.27353966, 0.12040212, 0.170736...","[-0.05627888, 0.07024487, -0.010127303, 0.0381..."
1,"Московская область, городской округ Пушкинский...",Акварель,5.0,Торговый центр,"Парковка удобная, есть на 1м этаже в самом ТЦ ...","[-0.5976521, -1.0532343, 0.47649938, -0.498017...","[0.044135116, 0.020764863, -0.018458415, -0.01...","[0.02728588, 0.008275842, -0.019191066, -0.015...","[0.47494435, -0.049932975, -0.0023330837, 0.08...","[-0.05162943, -0.102043465, -0.015492754, 0.04..."
2,"Псков, ул. Техническая / ул. Алексея Алёхина",Сити,5.0,Жилой комплекс,"Все нравится, кроме качества воды. Планировка ...","[-0.1946983, -1.0252259, 0.2142884, -0.5978085...","[-0.02027451, -0.03674739, 0.009350743, -0.041...","[-0.022215378, -0.027157225, 0.0011456553, -0....","[0.08608648, 0.013137772, -0.065643914, -0.034...","[-0.07400356, 0.006997604, -0.016366709, -0.05..."
3,"Тверь, проспект Победы, 3",Happy burger,5.0,Кафе Быстрое питание,"Туалет бесплатный, еду не пробовали, целью был...","[-0.63410574, -1.0250131, -0.010696959, -0.153...","[0.0050332267, 0.07289827, 0.007558107, 0.0310...","[-0.0302767, 0.050491307, 0.014834315, 0.02920...","[0.20306802, 0.048386086, -0.042062156, -0.137...","[0.10449143, 0.17446302, -0.015841514, -0.1183..."
4,"Ярославль, улица Кирова, 10/25",ЖаRим в КайF,4.0,Быстрое питание Кафе Ресторан,"Бургеры потрясающие, коктейли отличные, но над...","[-0.33710206, -0.7564586, -0.44192395, -0.1953...","[0.014471837, -0.014450337, 0.03563952, 0.0149...","[0.022243857, -0.013372567, 0.033722755, 9.218...","[0.07912045, -0.042511847, -0.13754338, 0.0355...","[-0.003875276, 0.11459597, -0.015715502, 0.012..."
...,...,...,...,...,...,...,...,...,...,...
19995,"Москва, Сокольническая площадь, 4А",Шоколадница,5.0,Кофейня Быстрое питание Кафе,"Приятное место, посещаем не первый год, были к...","[-0.5209945, -0.8757599, 0.40043816, -0.373585...","[0.02635513, 0.012479845, 0.004065965, -0.0071...","[0.018106937, 0.015331422, 0.0039730645, -0.01...","[0.10496237, -0.03670438, -0.07549223, 0.01840...","[-0.022670554, 0.028929016, -0.017267106, -0.0..."
19996,"Рязань, Московский район, улица Костычева, 9",Wildberries,5.0,Пункт выдачи,В пункте выдачи работают вежливые и отзывчивые...,"[-0.19544739, -0.4623036, 0.13189779, -0.41789...","[-0.009479776, 0.019591236, -0.02303938, -0.02...","[-0.008508198, 0.0081909755, -0.0037174106, -0...","[0.13070683, -0.0048493766, -0.14005418, 0.113...","[-0.03202977, -0.05333711, -0.016103536, 0.023..."
19997,Ростов-на-Дону,Сервис заказа такси Яндекс.Такси,1.0,Такси Курьерские услуги Автомобильные грузопер...,Хочу написать о работе сервиса Яндекс такси. М...,"[-0.501063, -0.98097295, 0.19489615, -0.211049...","[0.019867523, 0.0067250975, 0.0062512057, -0.0...","[0.0103744855, 0.007867257, 0.0057449033, -0.0...","[0.09571074, 0.09128091, 0.025380254, -0.04761...","[0.031879086, 0.10445784, -0.015782153, 0.0544..."
19998,"Московская область, Химки, Шереметьевское шосс...","Шереметьево, терминал C",5.0,Терминал аэропорта,В мартеапреле 2023 г. летели АрхангельскМосква...,"[-0.83474773, -0.39277935, 0.61578506, -0.5296...","[0.012428817, -0.025160825, -0.014360595, -0.0...","[-0.0006035709, -0.013560136, -0.0050858976, 0...","[0.2478286, -0.024608376, -0.14830263, -0.0258...","[-0.00417779, 0.0137610035, -0.015923437, -0.0..."


In [ ]:
model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

query = "Порекомендуй мне Японский ресторан"
query_emb = model.encode([query], convert_to_numpy=True)
embedding_matrix = np.vstack(df["text_sentence-transformers/paraphrase-multilingual-mpnet-base-v2"].values)
similarities = cosine_similarity(query_emb, embedding_matrix)[0]
top_indices = similarities.argsort()[-3:][::-1]
top_matches = df.iloc[top_indices].copy()

In [4]:
top_matches["text"].to_list()

['Ресторан могу смело рекомендовать абсолютно всем, кто любит отличный сервис, а также заведения с азиатской кухней, особо порадовал большой выбор еды и напитков. ',
 'Уютный ресторан с отличным обслуживанием и большим выбором очень вкусной ближневосточной еды👍👍👍 ',
 'Ресторан азиатской кухни  это место, которое стоит посетить каждому, кто ценит великолепный вкус и атмосферу Востока. Этот уютный, атмосферный ресторан, порадует вас не только своей многогранной кухней, но и великолепным интерьером.  Первое, что бросается в глаза, когда вы попадаете в ресторан азиатской кухни, это уютный интерьер, который перенесет вас в настоящий Восток. Музыка, образы на стенах и стильный декор создают неповторимую атмосферу, которая погружает вас в традиционную культуру страны.  А теперь о главном  о еде! Ресторан Азиатской кухни радует гурманов множеством блюд, при готовленных по классическим рецептам Азии. Здесь вы можете отведать свежие салаты, мясные блюда из говядины, свинины, курицы, а также блюд

In [ ]:
top_matches

In [ ]:
top_matches["similarity"] = similarities[top_indices]

# Display the results
print(top_matches[["data", "similarity"]])

In [3]:
df = df.sample(1000)

embedding_array = np.array(df["text_sentence-transformers/distiluse-base-multilingual-cased-v1"].tolist())

# Initialize and apply UMAP
reducer = umap.UMAP(n_components=2, random_state=42)
embedding_2d = reducer.fit_transform(embedding_array)

# Add reduced dimensions to the dataframe
df["x"] = embedding_2d[:, 0]
df["y"] = embedding_2d[:, 1]

/home/seara/Desktop/Github/travel-agent/.venv/lib/python3.13/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/seara/Desktop/Github/travel-agent/.venv/lib/python3.13/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [7]:
fig = px.scatter(
    df,
    x="x",
    y="y",
    color="rubrics",  # Replace with your actual hue column name
    title="UMAP Projection of Embeddings",
    # hover_data=df.columns,  # Shows more info on hover if desired
)

fig.show()

In [3]:
df = pd.read_csv("data/questions/embed_eval/normal_rubrics_15886.csv")
df["names_ru"] = df["names_ru"].str.split(";")

df_exploded = df.explode("names_ru", ignore_index=True)
df_exploded.columns = ["question", "name_ru"]
df_exploded

,question,name_ru
0,Где поесть вкусную пиццу?,People’s Food
1,Где поесть вкусную пиццу?,Big Rolls
2,Где поесть вкусную пиццу?,Casa Nostra
3,Где поесть вкусную пиццу?,Суши Wok
4,Где поесть вкусную пиццу?,Pizza Express
...,...,...
808,Магазины с посудой и товарами для кухни,Домовой
809,Магазины с посудой и товарами для кухни,Унидом
810,Магазины с посудой и товарами для кухни,Хозмир
811,Магазины с посудой и товарами для кухни,Всё для дома


In [4]:
raw = pd.read_csv("data/raw/geo-reviews-dataset-2023.csv")
raw

,address,name_ru,rating,rubrics,text
0,"Екатеринбург, ул. Московская / ул. Волгоградск...",Московский квартал,3.0,Жилой комплекс,Московский квартал 2.\nШумно : летом по ночам ...
1,"Московская область, Электросталь, проспект Лен...",Продукты Ермолино,5.0,Магазин продуктов;Продукты глубокой заморозки;...,"Замечательная сеть магазинов в общем, хороший ..."
2,"Краснодар, Прикубанский внутригородской округ,...",LimeFit,1.0,Фитнес-клуб,"Не знаю смутят ли кого-то данные правила, но я..."
3,"Санкт-Петербург, проспект Энгельса, 111, корп. 1",Snow-Express,4.0,Пункт проката;Прокат велосипедов;Сапсёрфинг,Хорошие условия аренды. \nДружелюбный персонал...
4,"Тверь, Волоколамский проспект, 39",Студия Beauty Brow,5.0,"Салон красоты;Визажисты, стилисты;Салон бровей...",Топ мастер Ангелина топ во всех смыслах ) Немн...
...,...,...,...,...,...
499995,"Москва, Южный административный округ, район Би...",Бирюлёво-Пассажирская,4.0,Железнодорожная станция,"Охрана кривая но добрая, двери не закрываются ..."
499996,"Москва, Южный административный округ, район Би...",Бирюлёво-Пассажирская,4.0,Железнодорожная станция,По сравнению со многими современными платформа...
499997,"Новосибирск, Коммунистическая улица, 48А",NaN,5.0,"Бар, паб","Приятная атмосфера, прекрасное вино, волшебная..."
499998,"Астраханская область, Харабалинский район",Сарай-Бату,5.0,Достопримечательность,Был с семьёй 13.06.23 Отличное место. Рекоменд...


In [5]:
df_exploded = df_exploded.merge(raw, how="left", on="name_ru")
df_exploded

,question,name_ru,address,rating,rubrics,text
0,Где поесть вкусную пиццу?,People’s Food,"Москва, Зеленоград, к100",5.0,Доставка еды и обедов;Пиццерия;Суши-бар,"Лучшая доставка👍быстро, вкусно, соотношение це..."
1,Где поесть вкусную пиццу?,People’s Food,"Москва, Зеленоград, к100",5.0,Доставка еды и обедов;Пиццерия;Суши-бар,Отличное место \nДружелюбный персонал\nХорошая...
2,Где поесть вкусную пиццу?,People’s Food,"Москва, Зеленоград, к100",5.0,Доставка еды и обедов;Пиццерия;Суши-бар,"Шикарная пицца, постоянно здесь заказываем, вс..."
3,Где поесть вкусную пиццу?,People’s Food,"Москва, Зеленоград, к100",5.0,Доставка еды и обедов;Пиццерия;Суши-бар,Всем доброго продуктивного дня!!Заказывю еду к...
4,Где поесть вкусную пиццу?,Big Rolls,"Пермь, Хабаровская улица, 56",5.0,Суши-бар;Пиццерия,"Шикарные роллы и очень вкусная пицца,всем оста..."
...,...,...,...,...,...,...
64116,Магазины с посудой и товарами для кухни,Всё для дома,"Москва, Свободный проспект, 33",5.0,Товары для дома;Магазин хозтоваров и бытовой х...,"Широкий ассортимент товаров на любой случай, м..."
64117,Магазины с посудой и товарами для кухни,Всё для дома,"Краснодарский край, Сочи, микрорайон Головинка...",5.0,Строительный магазин,Приятный и отзывчивый продавец. Довольно широк...
64118,Магазины с посудой и товарами для кухни,Всё для дома,"Московская область, Раменский городской округ,...",5.0,Магазин хозтоваров и бытовой химии,Хороший магазин в шаговой доступности. Всегда...
64119,Магазины с посудой и товарами для кухни,Посуда-центр,"Новосибирск, улица Мира, 61",5.0,Магазин посуды;Товары для дома,Были в том районе и решили зайти! Отличный маг...


In [6]:
df_exploded.to_parquet("data/embedding_bench/normal_rubrics_15886_exploded.parquet", index=False)

In [7]:
df_test = pd.read_csv("data/embedding/embedding_bench.csv")

In [9]:
(df_exploded == df_test).sum(0)

question    64121
name_ru     64121
address     64121
rating      64121
rubrics     64121
text        64121
dtype: int64

In [13]:
df = pd.read_parquet("data/embedding/moskva-with-embeddings.parquet")
df

,address,name_ru,rating,rubrics,text,text_DeepPavlov/rubert-base-cased-sentence,text_sentence-transformers/distiluse-base-multilingual-cased-v1,text_sentence-transformers/distiluse-base-multilingual-cased-v2,text_sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2,text_sentence-transformers/paraphrase-multilingual-mpnet-base-v2
0,"Москва, Каширское шоссе, 12А",Вкусно — и точка. Авто,5.0,Быстрое питание,"В плане приготовления, обслуживания все норма...","[-0.18565536, -0.52149856, 0.19672921, -0.3772...","[0.013134836, -0.025997657, -0.003641442, -0.0...","[0.0064481846, -0.016722756, -0.002055221, -0....","[0.050873335, -0.020492243, 0.10843523, -0.015...","[0.019578818, 0.027654123, -0.01855886, 0.0382..."
1,"Москва, Гольяновский сквер",Маяк,4.0,Жанровая скульптура,Всё красиво. Но огороженная территория работае...,"[-0.13149352, -0.97044003, 0.060365837, -0.319...","[0.0034782838, -0.0096344305, -0.013033644, -0...","[0.00198409, -0.010837117, -0.010307297, -0.01...","[0.24817377, -0.0016566084, 0.025207603, 0.010...","[-0.017705943, 0.09024238, -0.017515615, 0.065..."
2,"Москва, Северо-Восточный административный окру...",Laven,5.0,Салон красоты;Парикмахерская;Ногтевая студия,"Понравилась мастер маникюра Наталья, делает пр...","[-0.5854045, -0.30637515, 0.16740237, -0.38273...","[0.008512573, -0.02785064, -0.006967185, -0.00...","[0.0022515603, -0.027336042, 0.006761779, -0.0...","[0.0054059364, 0.08370612, 0.06217023, 0.07492...","[-0.046867177, -0.012386145, -0.016799098, 0.0..."
3,"Москва, Луков переулок, 10",La Joie,1.0,Ногтевая студия;Салон красоты;Салон бровей и р...,"К сожалению, осталась недовольна услугой, ногт...","[-0.5768657, -1.1830091, -0.013453014, -0.6032...","[-0.032284364, 0.020725776, 0.04147887, -0.035...","[-0.022766449, 0.009917881, 0.02834799, -0.027...","[-0.03897661, 0.16247645, 0.15520489, 0.195690...","[0.08584534, -0.013725076, -0.018957678, 0.077..."
4,"Москва, Украинский бульвар, 5",Брусника,5.0,Кафе;Магазин кулинарии;Кондитерская,Вкусно и хорошо. Цены реально приемлемы. Лучше...,"[-0.20414357, -0.57702947, -0.12487289, -0.564...","[0.014413389, -0.0070417086, -0.007830615, -0....","[0.015559374, -0.014255222, 0.0018718629, -0.0...","[0.16095647, -0.1648924, -0.013818163, 0.04877...","[0.019321434, 0.064687796, -0.014138108, 0.032..."
...,...,...,...,...,...,...,...,...,...,...
95,"Москва, Угрешская улица, 2, стр. 48",Антикор центр,5.0,Нанесение покрытий,"Спасибо! Всё профессионально, качественно, со ...","[-0.13890553, -0.6501167, -0.14960717, -0.1005...","[0.021223057, -0.016729247, -0.013730093, -0.0...","[0.019371778, -0.020500196, 0.0030716695, -0.0...","[-0.008642819, 0.0880089, 0.0140693, -0.040354...","[-0.03337363, 0.050874688, -0.017312814, 0.039..."
96,"Москва, Духовской переулок, 17, стр. 16",Мята Lounge,5.0,"Кальян-бар;Бар, паб;Ресторан","Тихое приятное место! Чистые кальяны, обходите...","[0.00671122, -0.6953245, -0.545217, -0.3377802...","[-0.010307218, -0.019970154, -0.009280098, -0....","[-0.009064786, -0.017014394, -0.005706454, -0....","[0.13068543, 0.09988778, 0.046770696, -0.03400...","[0.007199675, 0.053165194, -0.018950317, 0.022..."
97,"Москва, Беломорская улица, 9",Пятёрочка,2.0,Супермаркет,Помещение не очень персонал не очень и продукц...,"[-0.39335665, -1.5583383, -0.31729457, -0.9004...","[0.04100805, -0.026200088, 0.04560855, -0.0689...","[0.008408806, -0.028233202, 0.028427923, -0.04...","[0.26527318, 0.084578976, 0.12930866, -0.08996...","[0.07522322, 0.10483004, -0.016311519, -0.0306..."
98,"Москва, парк Свиблово",Смотровая площадка,5.0,Смотровая площадка,"Для прогулок классное место, совмещено с парко...","[-0.42783475, -1.1725686, 0.31510708, -0.67107...","[0.030313838, 0.0135219805, -0.010784422, 0.01...","[0.011584903, 0.005989259, 0.0049582063, -0.01...","[0.525896, 0.09654535, -0.055917013, 0.2008814...","[-0.09154773, 0.027200831, -0.018178415, 0.039..."


In [2]:
pd.set_option("max_colwidth", 400)
df = pd.read_csv("data/embedding_bench/normal_rubrics_15886_exploded_results.csv")
df_rounded = df.round(2)
markdown_output = df_rounded.to_markdown(index=False)
print(markdown_output)

| model                                                       |   map@1 |   map@3 |   map@5 |   map@10 |   map@20 |   embedding_duration_sec |   benchmark_duration_sec |   total_duration_sec |
|:------------------------------------------------------------|--------:|--------:|--------:|---------:|---------:|-------------------------:|-------------------------:|---------------------:|
| cointegrated/rubert-tiny2                                   |    0.19 |    0.21 |    0.19 |     0.18 |     0.16 |                    16.09 |                    11.24 |                27.32 |
| DeepPavlov/rubert-base-cased-sentence                       |    0.19 |    0.26 |    0.29 |     0.29 |     0.26 |                   207.21 |                    24.82 |               232.03 |
| ai-forever/sbert_large_nlu_ru                               |    0.23 |    0.26 |    0.27 |     0.27 |     0.24 |                   674.55 |                    32.81 |               707.37 |
| ai-forever/sbert_large_mt_nlu_ru 

In [1]:
df = pd.read_parquet("data/embedding/embeddings_moskva.parquet")
df

,address,name_ru,rating,rubrics,text,text_cointegrated/rubert-tiny2,text_DeepPavlov/rubert-base-cased-sentence,text_ai-forever/sbert_large_nlu_ru,text_ai-forever/sbert_large_mt_nlu_ru,text_sentence-transformers/distiluse-base-multilingual-cased-v1,text_sentence-transformers/distiluse-base-multilingual-cased-v2,text_sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2,text_sentence-transformers/paraphrase-multilingual-mpnet-base-v2,text_intfloat/multilingual-e5-large,text_intfloat/multilingual-e5-base,text_intfloat/multilingual-e5-small,text_ai-forever/ru-en-RoSBERTa,text_sergeyzh/BERTA
0,"Москва, 4-й Кожевнический переулок, 4",Jinju,5.0,Кафе;Кофейня,"5 из 5🖤 Пил кофе и в Риме, и в Париже, но вку...","[0.065158635, 0.027079074, -0.06572106, 0.0301...","[-0.63672847, -1.4398409, -0.24549058, -0.6321...","[0.01108983, -0.0026213285, -0.021376006, -0.0...","[0.0081150355, -0.0005738028, -0.08349785, -0....","[0.008887728, -0.038243067, 0.00077403017, -0....","[0.0026654408, -0.028006202, 0.0034495436, -0....","[0.11867878, -0.061117657, 0.080457725, 0.0813...","[0.040915422, 0.11509949, -0.01670815, 0.01367...","[0.025414402, -0.021630447, -0.017103506, -0.0...","[-0.015841102, 0.04712556, -0.028510993, 0.016...","[0.015238522, -0.00344009, -0.053025264, -0.03...","[-0.0155937, 0.02560155, 0.0004681375, 0.01960...","[0.000805113, -0.008762268, 0.009512121, 0.038..."
1,"Москва, 4-й Кожевнический переулок, 4",Jinju,4.0,Кафе;Кофейня,"Не очень удобное расположение, от метро идти м...","[0.10211347, 0.021025384, -0.037956174, -0.080...","[-0.7955147, -1.6629534, 0.1663701, -0.7365139...","[0.020285284, -0.022223586, -0.0066342833, -0....","[0.024190314, -0.00026632237, -0.054509725, -0...","[0.019696038, -0.03633522, 0.032869093, -0.030...","[-0.008414903, -0.054211285, 0.015036484, -0.0...","[0.29954767, -0.29195353, 0.09211391, 0.244134...","[0.05647306, 0.054179005, -0.016457029, -0.031...","[0.028021691, -0.023858428, -0.022046624, -0.0...","[0.02667061, 0.04104469, -0.04661966, 0.015832...","[-0.024401221, 0.010814473, -0.038674578, -0.0...","[-0.015493, 0.0013583833, 0.0031074146, -0.020...","[0.028762028, -0.00328872, 0.023196457, 0.0349..."
2,"Москва, улица Малая Дмитровка, 15",Healingbowl,5.0,Магазин подарков и сувениров;Музыкальный магазин,Заказывала чашу онлайн. Перед этим посмотрела ...,"[0.030677216, 0.0746183, -0.010435267, -0.0631...","[-0.7431688, -1.0197073, 0.7221362, -0.0677704...","[0.011211316, -0.024881674, 0.0009540356, -0.0...","[0.007983729, -0.0063359877, -0.016258711, -0....","[0.028080624, 0.070893735, 0.044071697, -0.023...","[0.014342122, 0.07502445, 0.011368184, -0.0504...","[-0.15994012, -0.12036769, -0.05947875, 0.0397...","[0.033403963, 0.011028338, -0.01706383, -0.002...","[0.013005017, -0.0042784484, -0.018139709, -0....","[0.0116824005, 0.060707282, -0.041346777, -0.0...","[0.0016556059, 0.018215127, -0.03693384, -0.05...","[0.0054352945, 0.012600248, -0.0057249893, 0.0...","[-0.03593939, -0.041930147, 0.020960586, 0.033..."
3,"Москва, улица Кирпичные Выемки, 2, корп. 1",Власть взгляда,5.0,Салон бровей и ресниц,Большое спасибо Миле за такие красивые и объём...,"[0.12854105, 0.04638249, -0.021692015, -0.0268...","[-0.4281867, -0.3352679, -0.168545, -0.1838862...","[0.048594404, 0.0032329562, -0.015679015, -0.0...","[0.031246524, 0.016939463, -0.05280223, -0.056...","[-0.0057994244, 0.00966078, -0.014317924, -0.0...","[0.01047921, -0.01214922, 0.01095549, -0.03239...","[0.006627138, -0.017381214, 0.1687543, 0.23530...","[-0.013692316, 0.10473775, -0.021663148, 0.044...","[0.024102708, -0.0032974442, -0.021706, -0.055...","[0.018364036, 0.051636506, -0.011785362, -0.00...","[-0.017720427, -0.01180869, -0.035164043, -0.0...","[0.004178859, 0.022546373, -0.016239543, -0.01...","[-0.02869467, -0.032135252, 0.028440826, 0.056..."
4,"Москва, Старокалужское шоссе, 62",Eastlines,5.0,Автомобильные грузоперевозки;Перевозка негабар...,"5 звезд вашему сервису! Очень понравилось, что...","[0.026958624, 0.00